In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 1. 함수

In [2]:
import os, re, json, gc, math
from typing import List, Dict, Any
import numpy as np
import pandas as pd
from datasets import Dataset
from dotenv import load_dotenv
load_dotenv()

from ragas import evaluate
from ragas.run_config import RunConfig

# -----------------------------
# 0) RAG pipeline
# -----------------------------
import sys
MODULES_PATH = os.path.join(os.getcwd(), "chatbot_app", "modules")
sys.path.append(MODULES_PATH)
sys.path.append(os.getcwd())
from chatbot_app.modules.rag_module import create_pipeline
try:
    from chatbot_app.modules.rag_module import RAGConfig
except Exception:
    RAGConfig = None

# -----------------------------
# 1) Metrics (ragas 버전 차이 대응)
# -----------------------------
try:
    from ragas.metrics._context_precision import ContextPrecision
except Exception:
    from ragas.metrics import context_precision as ContextPrecision

try:
    from ragas.metrics._context_recall import ContextRecall
except Exception:
    from ragas.metrics import context_recall as ContextRecall

try:
    from ragas.metrics._answer_relevancy import AnswerRelevancy
except Exception:
    from ragas.metrics import answer_relevancy as AnswerRelevancy

try:
    from ragas.metrics._faithfulness import Faithfulness
except Exception:
    from ragas.metrics import faithfulness as Faithfulness

def _metric_instance(m):
    return m() if callable(m) else m

CP_METRIC = _metric_instance(ContextPrecision)
CR_METRIC = _metric_instance(ContextRecall)
AR_METRIC = _metric_instance(AnswerRelevancy)
F_METRIC  = _metric_instance(Faithfulness)

# -----------------------------
# 2) LLM/Embeddings
#    - CP/CR: OpenAI (llm_factory)
#    - AR/F : Upstage (solar-pro2)
#    - Embeddings: UpstageEmbeddings(solar-embedding-1-large)
# -----------------------------
from openai import OpenAI
from ragas.llms import llm_factory
from langchain_upstage import ChatUpstage, UpstageEmbeddings

openai_client = OpenAI()
ragas_llm_openai = llm_factory(
    model="gpt-4o-mini",
    provider="openai",
    client=openai_client,
)

try:
    from ragas.llms import LangchainLLMWrapper
    ragas_llm_upstage = LangchainLLMWrapper(ChatUpstage(model="solar-pro2", temperature=0))
except Exception:
    ragas_llm_upstage = ChatUpstage(model="solar-pro2", temperature=0)

ragas_embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

# Upstage provider 제약: strictness=1 강제(AR에서 n=1 요구)
if hasattr(AR_METRIC, "strictness"):
    AR_METRIC.strictness = 1

# 실행 안정 설정
rc = RunConfig(max_workers=1, timeout=180)

# -----------------------------
# 3) AnswerRelevancy 세그폴트 회피 패치
# -----------------------------
try:
    import ragas.metrics._answer_relevance as ar_mod

    def _cos_sim(u, v, eps=1e-8):
        su = 0.0
        sv = 0.0
        s  = 0.0
        for a, b in zip(u, v):
            af = float(a); bf = float(b)
            s  += af * bf
            su += af * af
            sv += bf * bf
        return s / (math.sqrt(su) * math.sqrt(sv) + eps)

    def safe_calculate_similarity(self, question: str, generated_questions: list[str]):
        emb = None
        for name in ["embeddings", "_embeddings", "embedding", "_embedding"]:
            if hasattr(self, name):
                emb = getattr(self, name)
                if emb is not None:
                    break
        if emb is None:
            raise RuntimeError("AnswerRelevancy metric에 embeddings가 세팅되지 않았습니다.")

        def embed_one(text: str):
            if hasattr(emb, "embed_query"):
                return emb.embed_query(text)
            if hasattr(emb, "embed_documents"):
                return emb.embed_documents([text])[0]
            raise RuntimeError("embeddings 객체에 embed_query/embed_documents가 없습니다.")

        def embed_many(texts: list[str]):
            if len(texts) == 0:
                return []
            if hasattr(emb, "embed_documents"):
                return emb.embed_documents(texts)
            if hasattr(emb, "embed_query"):
                return [emb.embed_query(t) for t in texts]
            raise RuntimeError("embeddings 객체에 embed_query/embed_documents가 없습니다.")

        qv  = embed_one(question)
        gvs = embed_many(generated_questions)
        if not gvs:
            return np.array([0.0], dtype="float32")
        sims = [_cos_sim(qv, gv) for gv in gvs]
        return np.array(sims, dtype="float32")

    # 클래스 메서드 덮어쓰기
    if hasattr(ar_mod, "ResponseRelevancy"):
        ar_mod.ResponseRelevancy.calculate_similarity = safe_calculate_similarity
    if hasattr(ar_mod, "AnswerRelevancy"):
        ar_mod.AnswerRelevancy.calculate_similarity = safe_calculate_similarity

    if hasattr(ar_mod, "calculate_similarity"):
        ar_mod.calculate_similarity.__code__ = _cos_sim.__code__
        ar_mod.calculate_similarity.__defaults__ = _cos_sim.__defaults__

    if hasattr(AR_METRIC, "strictness"):
        AR_METRIC.strictness = 1

    print("✅ patched AnswerRelevancy similarity (pure python)")
except Exception as e:
    print("⚠️ AnswerRelevancy patch skipped:", repr(e))

# -----------------------------
# 4) Utils
# -----------------------------
OUT_DIR = "./data/RAGAS"
os.makedirs(OUT_DIR, exist_ok=True)

def cleanup_memory():
    gc.collect()

def validate_data(rows, required_fields):
    issues = []
    for i, r in enumerate(rows):
        for field in required_fields:
            if field not in r:
                issues.append(f"Row {i}: missing field '{field}'")
            elif r[field] is None:
                issues.append(f"Row {i}: field '{field}' is None")
            elif field in ["question", "answer"] and not str(r[field]).strip():
                issues.append(f"Row {i}: field '{field}' is empty")
    if issues:
        print("⚠️ Data validation issues:")
        for x in issues[:10]:
            print(" -", x)
    else:
        print("✅ Data validation passed")
    return len(issues) == 0

def clip_rows_cp(rows, *, max_ctx=3, max_chars=600):
    clipped = []
    for r in rows:
        rr = dict(r)
        ctx = rr.get("contexts", []) or []
        ctx = [c for c in ctx if c is not None]
        rr["contexts"] = [str(c)[:max_chars] for c in ctx[:max_ctx]]
        clipped.append(rr)
    return clipped

def clip_rows_cr(rows, *, max_ctx=2, max_chars=400, clip_reference_chars=600, clip_answer_chars=600):  # 200/300/300
    clipped = []
    for r in rows:
        rr = dict(r)
        ctx = rr.get("contexts", []) or []
        ctx = [c for c in ctx if c is not None]
        rr["contexts"] = [str(c)[:max_chars] for c in ctx[:max_ctx]]
        if clip_reference_chars is not None and rr.get("reference"):
            rr["reference"] = str(rr["reference"])[:clip_reference_chars]
        if clip_answer_chars is not None and rr.get("answer"):
            rr["answer"] = str(rr["answer"])[:clip_answer_chars]
        clipped.append(rr)
    return clipped

def clip_rows_af(rows, *, max_ctx=4, max_chars=1200, clip_answer_chars=700):
    clipped = []
    for r in rows:
        rr = dict(r)
        ctx = rr.get("contexts", []) or []
        ctx = [c for c in ctx if c is not None]
        rr["contexts"] = [str(c)[:max_chars] for c in ctx[:max_ctx]]
        if rr.get("answer"):
            rr["answer"] = str(rr["answer"])[:clip_answer_chars]
        clipped.append(rr)
    return clipped

# -----------------------------
# 5) DOCX -> items
# -----------------------------
from docx import Document as DocxDocument
DOCX_PATH = "./data/RAGAS/RAGAS_질문_10개_정리본.docx"

def parse_ragas_docx(doc_path: str) -> List[Dict[str, Any]]:
    doc = DocxDocument(doc_path)
    paras = [p.text.strip() for p in doc.paragraphs if p.text and p.text.strip()]
    text = "\n".join(paras)
    blocks = re.split(r"\n(?=\d+\.\s)", text)

    items = []
    for b in blocks:
        b = b.strip()
        if not re.match(r"^\d+\.\s", b):
            continue
        parts = b.split("✔️ 모범답안", 1)
        if len(parts) != 2:
            continue
        q_part = re.sub(r"^\d+\.\s*", "", parts[0].strip())
        a_part = parts[1].strip()
        items.append({"question": q_part.strip(), "ground_truth": a_part.strip()})

    if len(items) == 0:
        raise ValueError("docx에서 문항을 1개도 파싱하지 못했습니다.")
    return items

# -----------------------------
# 6) RAG 실행 -> rows (1회)
# -----------------------------
def run_rag_and_build_rows(items: List[Dict[str, Any]], pipeline) -> List[Dict[str, Any]]:
    rows = []
    for i, ex in enumerate(items, start=1):
        q = ex["question"]
        gt = ex.get("ground_truth", "")

        MAX_QUERY_CHARS = 2000
        q_for_retrieval = q if len(q) <= MAX_QUERY_CHARS else q[:MAX_QUERY_CHARS]

        try:
            trace = pipeline.answer_with_trace(q_for_retrieval, skip_normalization=False)
        except Exception as e:
            print(f"[ERROR] Q{i} retrieval failed:", repr(e))
            trace = {"answer": "", "docs": [], "normalized_query": ""}

        answer = trace.get("answer", "") or ""
        docs = trace.get("docs", []) or []

        contexts = []
        for d in docs:
            try:
                contexts.append(d.page_content)
            except Exception:
                pass

        rows.append({
            "id": i,
            "question": q,
            "answer": answer if answer else "답변을 생성할 수 없습니다.",
            "contexts": contexts,
            "reference": gt,
            "ground_truths": [gt],
            "normalized_query": trace.get("normalized_query", ""),
        })
    return rows

def to_dataset(rows: List[Dict[str, Any]]) -> Dataset:
    return Dataset.from_list([{
        "question": r["question"],
        "answer": r["answer"],
        "contexts": r.get("contexts", []),
        "reference": r.get("reference", ""),
    } for r in rows])

# -----------------------------
# 7) Main: 1 run -> total table
# -----------------------------
def run_once_total_table(
    cfg=None,
    out_csv: str = os.path.join(OUT_DIR, "ragas_total_score_table.csv"),
):
    items = parse_ragas_docx(DOCX_PATH)
    print("✅ items loaded:", len(items))

    # pipeline 생성
    if cfg is not None:
        pipeline = create_pipeline(config=cfg)
    else:
        pipeline = create_pipeline()

    rows_local = run_rag_and_build_rows(items, pipeline)
    validate_data(rows_local, required_fields=["question", "answer", "contexts"])

    # 지표별 클립
    rows_cp = clip_rows_cp(rows_local, max_ctx=3, max_chars=600)
    rows_cr = clip_rows_cr(rows_local, max_ctx=2, max_chars=200, clip_reference_chars=300, clip_answer_chars=300)
    rows_af = clip_rows_af(rows_local, max_ctx=4, max_chars=1200, clip_answer_chars=700)

    ds_cp = to_dataset(rows_cp)
    ds_cr = to_dataset(rows_cr)
    ds_af = to_dataset(rows_af)

    # CP (OpenAI)
    res_cp = evaluate(
        dataset=ds_cp,
        metrics=[CP_METRIC],
        llm=ragas_llm_openai,
        embeddings=ragas_embeddings,
        run_config=rc,
        show_progress=True,
        raise_exceptions=True,
        batch_size=1,
    )
    df_cp = res_cp.to_pandas()
    if "id" not in df_cp.columns:
        df_cp["id"] = range(1, len(df_cp) + 1)

    # CR (OpenAI)
    res_cr = evaluate(
        dataset=ds_cr,
        metrics=[CR_METRIC],
        llm=ragas_llm_openai,
        embeddings=ragas_embeddings,
        run_config=rc,
        show_progress=True,
        raise_exceptions=True,
        batch_size=1,
    )
    df_cr = res_cr.to_pandas()
    if "id" not in df_cr.columns:
        df_cr["id"] = range(1, len(df_cr) + 1)

    # AR/F (Upstage)
    if hasattr(AR_METRIC, "strictness"):
        AR_METRIC.strictness = 1

    res_af = evaluate(
        dataset=ds_af,
        metrics=[AR_METRIC, F_METRIC],
        llm=ragas_llm_upstage,
        embeddings=ragas_embeddings,
        run_config=rc,
        show_progress=True,
        raise_exceptions=True,
        batch_size=1,
    )
    df_af = res_af.to_pandas()
    if "id" not in df_af.columns:
        df_af["id"] = range(1, len(df_af) + 1)

    # 통합 테이블
    merged = (
        df_cp[["id", "context_precision"]]
        .merge(df_cr[["id", "context_recall"]], on="id", how="inner")
        .merge(df_af[["id", "answer_relevancy", "faithfulness"]], on="id", how="inner")
        .sort_values("id")
    )

    merged = merged.rename(columns={
        "context_precision": "context-precision",
        "context_recall": "context-recall",
        "answer_relevancy": "answer-relevancy",
        "faithfulness": "faithfulness",
    })

    score_cols = ["context-precision", "context-recall", "answer-relevancy", "faithfulness"]
    for c in score_cols:
        merged[c] = pd.to_numeric(merged[c], errors="coerce")

    # Ri = 0.4*CP + 0.6*CR (retrieval integrated score)
    merged["Ri"] = 0.4 * merged["context-precision"] + 0.6 * merged["context-recall"]

    # 소수점 2자리 고정 (행들도 2자리)
    merged[score_cols + ["Ri"]] = merged[score_cols + ["Ri"]].round(2)

    # mean 행(2자리)
    mean_row = {"id": "mean"}
    for c in score_cols + ["Ri"]:
        mean_row[c] = round(float(merged[c].mean(skipna=True)), 2)

    out_df = pd.concat([merged, pd.DataFrame([mean_row])], ignore_index=True).set_index("id")

    os.makedirs(os.path.dirname(out_csv), exist_ok=True)
    out_df.to_csv(out_csv, encoding="utf-8")
    print("✅ saved:", out_csv)

    cleanup_memory()
    return out_df, out_csv

C:\Users\Admin\AppData\Local\Temp\ipykernel_13116\865369951.py:41: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import answer_relevancy as AnswerRelevancy


✅ patched AnswerRelevancy similarity (pure python)


C:\Users\Admin\AppData\Local\Temp\ipykernel_13116\865369951.py:75: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  ragas_llm_upstage = LangchainLLMWrapper(ChatUpstage(model="solar-pro2", temperature=0))


# 실행

In [3]:
%%time

if __name__ == "__main__":
    cfg = RAGConfig(
        k_law=7, 
        k_rule=7, 
        k_case=3, 
        search_multiplier=4, 
        
        enable_rerank=True,
        
        bm25_algorithm="okapi", # "okapi" | "plus"
        bm25_k1=1.5, 
        bm25_max_doc_chars=2200,
        
        hybrid_sparse_title_ratio=0.35,
        hybrid_dense_weight=0.5, 
        hybrid_sparse_weight=0.5, 
        
        rerank_threshold=0.2,
        rerank_max_documents=22,
        rerank_doc_max_chars=2600, 
        )
    table, path = run_once_total_table(cfg=cfg)

2026-02-06 14:42:02,627 - chatbot_app.modules.rag_module - INFO - 🔗 Pinecone 3중 인덱스 연결 중...


✅ items loaded: 10


2026-02-06 14:42:05,340 - chatbot_app.modules.rag_module - INFO - ✅ [Law / Rule / Case] 3개 인덱스 로드 완료!
2026-02-06 14:42:05,385 - chatbot_app.modules.rag_module - INFO - ℹ️ SimpleTokenizer 사용 (BM25)
2026-02-06 14:42:06,587 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-02-06 14:42:06,602 - chatbot_app.modules.rag_module - INFO - 🔄 표준화된 질문: 주민등록·확정일자 했는데, 확정일자부 내용(확정일자부 내용)이 중요한가요?  

[변경된 질문]  
주민등록(주민등록)·확정일자(확정일자) 했는데, 확정일자부 내용(확정일자부 내용)이 중요한가요?  

※ 참고: '확정일자부 내용'은 용어 사전에 없는 표현이므로 원문을 유지하였습니다. 필요 시 추가 매핑 요청 바랍니다.
2026-02-06 14:42:06,603 - chatbot_app.modules.rag_module - INFO - 🔍 [Hybrid Retrieval] query='주민등록·확정일자 했는데, 확정일자부 내용(확정일자부 내용)이 중요한가요?  

[변경된 질문]  
주민등록(주민등록)·확정일자(확정일자) 했는데, 확정일자부 내용(확정일자부 내용)이 중요한가요?  

※ 참고: '확정일자부 내용'은 용어 사전에 없는 표현이므로 원문을 유지하였습니다. 필요 시 추가 매핑 요청 바랍니다.'
2026-02-06 14:42:07,046 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-06 14:42:09,418

2026-02-06 14:42:36,144 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-06 14:42:36,956 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-06 14:42:37,447 - httpx - INFO - HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 429 Too Many Requests"
2026-02-06 14:42:37,447 - chatbot_app.modules.rag_module - WARNING - ⚠️ Rerank 실패 (skip): headers: {'access-control-expose-headers': 'X-Debug-Trace-ID', 'cache-control': 'no-cache, no-store, no-transform, must-revalidate, private, max-age=0', 'content-encoding': 'gzip', 'content-type': 'application/json', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'pragma': 'no-cache', 'vary': 'Origin,Accept-Encoding', 'x-accel-expires': '0', 'x-debug-trace-id': 'b95a5ad6957104b28717a3346469a418', 'x-trial-endpoint-call-limit': '40', 'x-trial-endpoint-call-remaining': '36', 'date': 'Fri, 06 Feb 2026 05:42:37 GMT', 'x-envoy-upstream-service-

2026-02-06 14:43:05,218 - chatbot_app.modules.rag_module - WARNING - ⚠️ Rerank 실패 (skip): headers: {'access-control-expose-headers': 'X-Debug-Trace-ID', 'cache-control': 'no-cache, no-store, no-transform, must-revalidate, private, max-age=0', 'content-encoding': 'gzip', 'content-type': 'application/json', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'pragma': 'no-cache', 'vary': 'Origin,Accept-Encoding', 'x-accel-expires': '0', 'x-debug-trace-id': '0dcafcd2ffda10961acb20e4d20149d2', 'x-trial-endpoint-call-limit': '40', 'x-trial-endpoint-call-remaining': '33', 'date': 'Fri, 06 Feb 2026 05:43:04 GMT', 'x-envoy-upstream-service-time': '7', 'server': 'envoy', 'via': '1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'transfer-encoding': 'chunked'}, status_code: 429, body: {'id': '029f7962-e513-4611-a2b9-a108df13a408', 'message': "You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Produ

...

2026-02-06 14:45:05,404 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-06 14:45:15,091 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-06 14:45:26,165 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-06 14:45:34,993 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-06 14:45:43,920 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-06 14:45:53,434 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-06 14:46:02,264 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-06 14:46:11,168 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-06 14:46:18,128 - httpx 

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

Batch 1/20:   0%|          | 0/1 [00:00<?, ?it/s]

2026-02-06 14:46:26,680 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-02-06 14:46:27,763 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-06 14:46:29,103 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-06 14:46:30,793 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-02-06 14:46:37,036 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-02-06 14:46:38,295 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-02-06 14:46:39,402 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-06 14:46:40,213 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-

✅ saved: ./data/RAGAS\ragas_total_score_table.csv
CPU times: total: 8.02 s
Wall time: 6min 26s


In [4]:
table = pd.read_csv('data/RAGAS/ragas_total_score_table.csv')
table

,id,context-precision,context-recall,answer-relevancy,faithfulness
0,1,1.0,0.857143,0.413251,0.400000
1,2,1.0,0.142857,0.521555,0.142857
2,3,1.0,0.875000,0.509518,0.222222
3,4,1.0,0.555556,0.438959,0.363636
4,5,1.0,1.000000,0.252470,0.000000
5,6,1.0,1.000000,0.364895,0.428571
6,7,1.0,0.400000,0.445392,0.100000
7,8,1.0,0.400000,0.508144,0.500000
8,9,1.0,0.500000,0.317246,0.222222
9,10,1.0,0.750000,0.403050,0.285714
